In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse





DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)

def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))



In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
A = np.array(nx.adjacency_matrix(g).todense(), dtype = np.float64)
deg_array = np.array(list(dict(g.degree).values()))

D_inv_A = np.zeros_like(A)

for i in range(num_nodes):
    D_inv_A[i] = A[i] / deg_array[i]


In [7]:
multi_hop_A = np.linalg.matrix_power(D_inv_A, 2)
# torch.save(multi_hop_A, "A_2hop.pkl")
# multi_hop_A = torch.load("A_2hop.pkl")

In [18]:
# set diagonal of 2-hop adjacency to 0
for i in range(num_nodes):
    multi_hop_A[i, i] = 0

In [24]:
# for GATE estimation
(np.matmul(multi_hop_A, np.ones((num_nodes, 1))).mean() )

0.9781957975786836

In [25]:
(np.matmul(multi_hop_A, np.ones((num_nodes, 1))).mean() ) * 0.5

0.4890978987893418

In [10]:
node_list = list(g.nodes.keys())

In [27]:
def po_2hop_linear_model(graph, z_vec, alpha=1, beta=1, sigma=0.1, gamma=1, r1=1, r2=0.5):        
    y_vec = alpha + beta * z_vec + sigma * np.random.normal(size=(num_nodes, 1)) + gamma * (
    r1 * np.matmul(D_inv_A, z_vec) + r2 * np.matmul(multi_hop_A, z_vec)
)
    
    for i in range(num_nodes):
        graph.nodes[node_list[i]]["y"] = y_vec[i][0]
        graph.nodes[node_list[i]]["z"] = z_vec[i][0]

In [33]:
node_list = np.array(node_list)

# from node to index in node_list
node_list_index_dict = {}
for node in g.nodes:
    node_list_index_dict[node] = np.where(node_list == node)[0][0]

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

loss_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")        
        torch.manual_seed(2)
        p_list = ramps[num_step:]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z_vector = np.zeros((num_nodes)) 
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            tr_units = list(reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters]))
            for node in tr_units:
                z_vector[node_list_index_dict[node]] = 1
                
            po_2hop_linear_model(g, z_vector.reshape(-1,1)) # add dimension
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        loss_list[seed, num_step] = loss.item()
        
        
    torch.save(GATE_hat_list, "Result/2hop_CR_cluster.pkl")
    torch.save(loss_list, "2hop_loss_list.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 8.0729
Epoch 0, Treat Prop: 0.05, Loss: 2.4004
Epoch 0, Treat Prop: 0.10, Loss: 1.3632
Epoch 0, Treat Prop: 0.25, Loss: 2.1167
Epoch 0, Treat Prop: 0.50, Loss: 4.9528
Epoch 50, Treat Prop: 0.02, Loss: 0.1494
Epoch 50, Treat Prop: 0.05, Loss: 0.2010
Epoch 50, Treat Prop: 0.10, Loss: 0.1613
Epoch 50, Treat Prop: 0.25, Loss: 0.2851
Epoch 50, Treat Prop: 0.50, Loss: 1.0701
Epoch 100, Treat Prop: 0.02, Loss: 0.0365
Epoch 100, Treat Prop: 0.05, Loss: 0.0344
Epoch 100, Treat Prop: 0.10, Loss: 0.0364
Epoch 100, Treat Prop: 0.25, Loss: 0.0634
Epoch 100, Treat Prop: 0.50, Loss: 0.0997
Epoch 150, Treat Prop: 0.02, Loss: 0.0329
Epoch 150, Treat Prop: 0.05, Loss: 0.0319
Epoch 150, Treat Prop: 0.10, Loss: 0.0298
Epoch 150, Treat Prop: 0.25, Loss: 0.0626
Epoch 150, Treat Prop: 0.50, Loss: 0.0829
Epoch 200, Treat Prop: 0.02, Loss: 0.0356
Epoch 200, Treat Prop: 0.05, Loss: 0.0317
Epoch 200, Treat Prop: 0.10, Loss: 0.0296
Epoch 200, Treat Prop: 0.25, Loss: 0.0659
Epoch 2

  0%|          | 1/1000 [00:30<8:29:57, 30.63s/it]

...

Epoch 350, Treat Prop: 0.50, Loss: 0.0511
0.05037529021501541
Seed: 385
Epoch 0, Treat Prop: 0.02, Loss: 7.8918
Epoch 0, Treat Prop: 0.05, Loss: 2.3602
Epoch 0, Treat Prop: 0.10, Loss: 2.2994
Epoch 0, Treat Prop: 0.25, Loss: 3.3940
Epoch 0, Treat Prop: 0.50, Loss: 5.0378
Epoch 50, Treat Prop: 0.02, Loss: 0.2267
Epoch 50, Treat Prop: 0.05, Loss: 0.2939
Epoch 50, Treat Prop: 0.10, Loss: 0.1866
Epoch 50, Treat Prop: 0.25, Loss: 0.2856
Epoch 50, Treat Prop: 0.50, Loss: 0.5300
Epoch 100, Treat Prop: 0.02, Loss: 0.0363
Epoch 100, Treat Prop: 0.05, Loss: 0.0475
Epoch 100, Treat Prop: 0.10, Loss: 0.0607
Epoch 100, Treat Prop: 0.25, Loss: 0.0754
Epoch 100, Treat Prop: 0.50, Loss: 0.0882
Epoch 150, Treat Prop: 0.02, Loss: 0.0240
Epoch 150, Treat Prop: 0.05, Loss: 0.0289
Epoch 150, Treat Prop: 0.10, Loss: 0.0529
Epoch 150, Treat Prop: 0.25, Loss: 0.0663
Epoch 150, Treat Prop: 0.50, Loss: 0.0911
Epoch 200, Treat Prop: 0.02, Loss: 0.0321
Epoch 200, Treat Prop: 0.05, Loss: 0.0252
Epoch 200, Treat Pr

 39%|███▊      | 386/1000 [2:42:53<4:17:41, 25.18s/it]

0.049796659499406815
Seed: 386
Epoch 0, Treat Prop: 0.02, Loss: 7.9002
Epoch 0, Treat Prop: 0.05, Loss: 2.3719
Epoch 0, Treat Prop: 0.10, Loss: 1.5515
Epoch 0, Treat Prop: 0.25, Loss: 2.3305
Epoch 0, Treat Prop: 0.50, Loss: 4.5717
Epoch 50, Treat Prop: 0.02, Loss: 0.1567
Epoch 50, Treat Prop: 0.05, Loss: 0.2045
Epoch 50, Treat Prop: 0.10, Loss: 0.1725
Epoch 50, Treat Prop: 0.25, Loss: 0.2680
Epoch 50, Treat Prop: 0.50, Loss: 0.7782
Epoch 100, Treat Prop: 0.02, Loss: 0.0280
Epoch 100, Treat Prop: 0.05, Loss: 0.0429
Epoch 100, Treat Prop: 0.10, Loss: 0.0354
Epoch 100, Treat Prop: 0.25, Loss: 0.0546
Epoch 100, Treat Prop: 0.50, Loss: 0.1080
Epoch 150, Treat Prop: 0.02, Loss: 0.0257
Epoch 150, Treat Prop: 0.05, Loss: 0.0378
Epoch 150, Treat Prop: 0.10, Loss: 0.0311
Epoch 150, Treat Prop: 0.25, Loss: 0.0581
Epoch 150, Treat Prop: 0.50, Loss: 0.0923
Epoch 200, Treat Prop: 0.02, Loss: 0.0307
Epoch 200, Treat Prop: 0.05, Loss: 0.0380
Epoch 200, Treat Prop: 0.10, Loss: 0.0312
Epoch 200, Treat P

 39%|███▊      | 387/1000 [2:43:18<4:15:23, 25.00s/it]

0.05422140285372734
Seed: 387
Epoch 0, Treat Prop: 0.02, Loss: 8.0565
Epoch 0, Treat Prop: 0.05, Loss: 2.4728
Epoch 0, Treat Prop: 0.10, Loss: 1.4582
Epoch 0, Treat Prop: 0.25, Loss: 2.0014
Epoch 0, Treat Prop: 0.50, Loss: 3.7546
Epoch 50, Treat Prop: 0.02, Loss: 0.1388
Epoch 50, Treat Prop: 0.05, Loss: 0.2035
Epoch 50, Treat Prop: 0.10, Loss: 0.1563
Epoch 50, Treat Prop: 0.25, Loss: 0.2428
Epoch 50, Treat Prop: 0.50, Loss: 0.7080
Epoch 100, Treat Prop: 0.02, Loss: 0.0282
Epoch 100, Treat Prop: 0.05, Loss: 0.0366
Epoch 100, Treat Prop: 0.10, Loss: 0.0322
Epoch 100, Treat Prop: 0.25, Loss: 0.0542
Epoch 100, Treat Prop: 0.50, Loss: 0.0831
Epoch 150, Treat Prop: 0.02, Loss: 0.0319
Epoch 150, Treat Prop: 0.05, Loss: 0.0310
Epoch 150, Treat Prop: 0.10, Loss: 0.0304
Epoch 150, Treat Prop: 0.25, Loss: 0.0574
Epoch 150, Treat Prop: 0.50, Loss: 0.0746
Epoch 200, Treat Prop: 0.02, Loss: 0.0331
Epoch 200, Treat Prop: 0.05, Loss: 0.0293
Epoch 200, Treat Prop: 0.10, Loss: 0.0301
Epoch 200, Treat Pr